In [3]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

In [3]:
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

documents = LlamaParse(result_type="markdown")


def pdf_parser(pdf_file_path: str):
    """
    PDF 파일을 파싱하여 그 내용을 Markdown 파일로 저장합니다.

    Args:
        pdf_file_path (str): 처리할 PDF 파일의 경로.
    """
    print(f"🔄 '{pdf_file_path}' 파일 파싱을 시작합니다...")

    try:
        # parsing instruction 을 지정합니다.
        parsing_instruction = (
            "You are parsing a AI Report. Please extract tables in markdown format."
        )

        # LlamaParse 설정
        parser = LlamaParse(
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
            result_type="markdown",
            # parsing_mode="Unstructured",
            language="ko",
            parsing_instruction=parsing_instruction,
        )

        # 1. LlamaParse를 사용하여 PDF 파일을 로드합니다.
        # 'documents' 객체는 이 함수 외부에서 미리 정의되어 있어야 합니다.
        parsed_docs = documents.load_data(file_path=pdf_file_path)

        # 2. LangChain 형식의 도큐먼트로 변환합니다.
        docs = [doc.to_langchain_format() for doc in parsed_docs]

        # 3. 저장할 Markdown 파일의 경로를 생성합니다. (확장자 변경)
        file_root, _ = os.path.splitext(pdf_file_path)
        output_file_path = file_root + ".md"

        # 4. 모든 페이지의 내용을 하나의 텍스트로 합칩니다.
        #    페이지 사이는 두 줄로 띄어 가독성을 높입니다.
        full_text = "\n\n".join([doc.page_content for doc in docs])

        # 5. 추출된 전체 텍스트를 .md 파일로 저장합니다.
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        print(f"✅ 파일 저장 완료: {output_file_path}")

    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다 - {pdf_file_path}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


# --- 함수 사용 예시 ---
# 이 코드를 실행하기 전에 'documents' 파서 객체를 초기화해야 합니다.
# file_to_parse = "data/디지털정부혁신추진계획.pdf"
file_to_parse = "./data/lorem-ipsum-10pages.pdf"
pdf_parser(file_to_parse)

🔄 './data/lorem-ipsum-10pages.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id c125c980-fc86-4be9-bafc-ce21d9c7959f
✅ 파일 저장 완료: ./data/lorem-ipsum-10pages.md


In [4]:
import os
import asyncio
from llama_cloud_services import LlamaParse

# --- 1. Llama Cloud API 키 설정 ---
# https://cloud.llamaindex.ai/ 에서 발급받은 API 키를 환경 변수에 설정해야 합니다.
if "LLAMA_CLOUD_API_KEY" not in os.environ:
    # 아래 주석을 해제하고 실제 키를 입력하거나, 시스템 환경 변수를 설정하세요.
    # os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..." 
    print("경고: 'LLAMA_CLOUD_API_KEY' 환경 변수가 설정되지 않았습니다.")

# --- 2. 파싱할 PDF 파일 경로 ---
pdf_file_path = "./data/클라우드네이티브1-3.pdf" # 실제 파일 경로로 수정하세요.

# --- 3. LlamaParse 객체에 고급 옵션 설정 ---
# 요청하신 모든 옵션을 포함하여 파서를 설정합니다.
# result_mode="markdown"이 최종 출력을 마크다운으로 지정하는 핵심 부분입니다.
parser = LlamaParse(
    result_mode="markdown",                # 최종 결과 형식을 마크다운으로 지정
    parse_mode="parse_page_with_agent",    # 에이전트 기반의 페이지별 파싱 모드 사용
    model="openai-gpt-4o-mini",            # 파싱에 사용할 LLM 모델 (gpt-4-1-mini -> gpt-4o-mini로 변경 권장)
    language="ko",
    high_res_ocr=True                      # 고해상도 OCR 활성화 (스캔 문서 처리에 유용)
)

# --- 4. 비동기 함수로 파일 파싱 실행 ---
async def parse_document():
    if "LLAMA_CLOUD_API_KEY" not in os.environ:
        print("API 키가 없어 파싱을 진행할 수 없습니다.")
        return

    print(f"'{pdf_file_path}' 파일 파싱을 시작합니다...")
    try:
        # aload_data 메서드를 사용하여 비동기적으로 파일을 파싱합니다.
        documents = await parser.aload_data(file_path=pdf_file_path)

        # 파싱된 마크다운 결과를 출력합니다.
        if documents:
            print("\n--- 파싱 결과 (마크다운) ---")
            print(documents[0].text)
            print("--------------------------")
        else:
            print("문서에서 내용을 추출하지 못했습니다.")
            
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

# --- 5. 비동기 함수 실행 ---
if __name__ == "__main__":
    asyncio.run(parse_document())

'./data/클라우드네이티브1-3.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id a77a70e0-b789-468e-b051-42cbced99d8a

--- 파싱 결과 (마크다운) ---
 플라우드 네이티브 아키택처의 개요

 단순히 플라우드 환경에서 애플리켜이선올 호스팅하는 것올 넘어서  플라우드의 특성올 최대한 활용하여 애플리키이선올
 설계하고 개발하는 방식울 의미한다. 플라우드 네이티브 아키택처는 이러한 접근 방식울 체계적으로 구현하기 위한 방법론
 과 원직올 제시한다

 플라우드 네이티브   아키택처의 핵심 요소는                    자동화

 로서비스는 애플리켜이선올 독립적으로 배포 가능한 작은 서비스로 나누는 접근 방식으로 각 서비스는 특정 기능울 수행한
 다 이로 인해 개발림은 각 서비스에 대해 독립적으로 개발 테스트 및 배포할 수 있으며 장애 발생 시 전체 시스템에 영향울
 미치지 않도록 할 수 있다

 건테이너 기술은 이러한 마이크로서비스트 배포하고 관리하는 데 중요한 역할울 한다 건테이너는 애플리켜이선과 그 실행
 에 필요한 모든 종속성올 패키장하여 손쉽게 배포할 수 잇도록 해준다: 이논 개발환경과 운영환경 간의 불일치률 줄이고 애

 여 애플리켜이선의 배포와 관리틀 자동화하고 확장성올 용이하게 한다

자동화는 플라우드 네이티브 아키택처의 또 다른 중요한 요소로 배포 테스트 모니터랑 등의 여러 프로세스틀 자동화하여
개발팀의 생산성흘 높인다: 이틀 통해 개발자는 반복적인 작업에 소요되는 시간을 줄이고 더 나아가 학신적인 기능울 개발
하늘 데 집중할 수 있다: 디-디는 이러한 자동화의 대표적인 예로 코드의 변경 사항이 발생활 때마다 자동으로 테스트하고
배포하는 프로세스름 통해 신속하게 기능울 제공활 수 있다:

 플라우드 네이티브 아키택처의 또 다른 장점은 스레일림과 유연성이다: 플라우드 환경은 사용자가 필요에 따라 자원올 동적
으로 조정할 수 있는 유연성올 제공하다: 이논